In [ ]:
#import tensorflow.keras as keras
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
#from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix, mean_squared_error, mean_squared_log_error, classification_report, balanced_accuracy_score

from sklearn.utils import class_weight
#from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from xgboost import XGBClassifier

import optuna
from optuna.samplers import TPESampler

#import imblearn
from imblearn.over_sampling import SMOTE

from tensorflow.keras.losses import CategoricalCrossentropy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
#sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')
test.head()

In [ ]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

# counts each type of Class
sorted(train['target'].value_counts())

In [ ]:
lencoder = LabelEncoder()
target = pd.DataFrame(lencoder.fit_transform(train['target']),columns=['target'])

train.drop(['target'], inplace=True, axis=1)

In [ ]:
sns.countplot(x = 'target', data = target)

In [ ]:
#oversample = SMOTE()
#trainS, targetS = oversample.fit_resample(train, target)
#np.sum(targetS, axis = 0)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size = 0.20, 
                                                  stratify = target, random_state = 2021)

In [ ]:
# Taking from https://www.kaggle.com/remekkinas/tps-5-weighted-training-xgb-rf-lr-smote

def training(model, X_train_oof, y_train_oof, weighted = False, b_type = True):
    test_preds = None
    test_oof_preds = None
    train_rmse = 0
    val_rmse = 0
    n_splits = 5
    
    skf = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = 4042)
    for fold, (tr_index, val_index) in enumerate(skf.split(X_train_oof.values, y_train_oof.values)):
        
        print(f"\nFold {fold + 1}")
        x_train_o, x_val_o = X_train_oof.iloc[tr_index], X_train_oof.iloc[val_index]
        y_train_o, y_val_o = y_train_oof.iloc[tr_index], y_train_oof.iloc[val_index]
        
        if weighted:
            weights_y = weights_df.iloc[tr_index]

        eval_set = [(x_val_o, y_val_o)]
        
        if b_type:
            if weighted:
                model.fit(x_train_o, y_train_o, eval_set = eval_set, verbose = 500, sample_weight = weights_y)
            else:
                model.fit(x_train_o, y_train_o, eval_set = eval_set, verbose = 500)
        
        else:
            model.fit(x_train_o, y_train_o)

        train_preds = model.predict(x_train_o)
        train_rmse += mean_squared_error(y_train_o,train_preds, squared = False)
        print("\n- Training RMSE : ", mean_squared_error(y_train_o,train_preds, squared = False))

        val_preds = model.predict(x_val_o)
        val_rmse += mean_squared_error(y_val_o, val_preds, squared = False)
        print("- Validation RMSE : ", mean_squared_error(y_val_o, val_preds, squared = False))
        print('---------------')

        if test_preds is None:
            test_preds = model.predict_proba(test.values)
            test_oof_preds = model.predict_proba(X_val.values)
        else:
            test_preds += model.predict_proba(test.values)
            test_oof_preds += model.predict_proba(X_val.values)

    print("\nAverage Training RMSE : " , train_rmse / n_splits)
    print("Average Validation RMSE : " , val_rmse / n_splits)

    test_preds /= n_splits
    test_oof_preds /= n_splits
    
    return test_preds, test_oof_preds

In [ ]:
# sample weights over class weigths (b/c of LabelEncoder?)
# If SMOTE then weigthed = False. Or not?

from sklearn.utils.class_weight import compute_sample_weight
weights_df = pd.DataFrame(compute_sample_weight("balanced", y_train.target), columns = ['weight'])

xgb_model_weighted = xgb.XGBClassifier(eval_metric='mlogloss')
test_preds, y_pred = training(xgb_model_weighted, X_train, y_train, weighted = True)

In [ ]:
y_preds = np.argmax(y_pred, axis=1)
print(f'MSE Score: {mean_squared_error(y_val,y_preds)}\n')
print(classification_report(y_val, y_preds))

sns.heatmap(pd.DataFrame(confusion_matrix(y_val, y_preds)),annot = True, linewidths =.5, fmt = "d")

In [ ]:
sns.countplot(x = 'target', data= pd.DataFrame(y_preds, columns=['target']))

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

sample_submission[['Class_1','Class_2', 'Class_3', 'Class_4']] = test_preds

sample_submission.to_csv("my_submissionSW.csv",index = False)
sample_submission.head(10)